# Code to compare different M1M3 settings.  See SITCOM-1160.

Craig Lage 26-Dec-23

In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import openpyxl as xl
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient
from lsst.ts.xml.tables.m1m3 import FATable, FAIndex, force_actuator_from_id, actuator_id_to_index

In [ ]:
"""
From this confluence site
https://confluence.lsstcorp.org/pages/viewpage.action?spaceKey=MPMIS&title=M1M3+inertial+forces+tests+-+timestamps+of+interest
"""
tests = [["2023-12-15T23:24", "2023-12-15T23:40", "40%", "red-red-red-green, e.g. no compensations"], 
         ["2023-12-15T23:49", "2023-12-16T00:05", "40%", "red-green-red-green - HP load cells only, balance forces"], 
         ["2023-12-19T02:32", "2023-12-19T02:45", "40%", "green-red-green-green - DC Accelerometers, TMA velocity"], 
         ["2023-12-20T08:08", "2023-12-20T08:21", "40%", "red-red-red-green, testing PID freeze without compensation, TMA faulted"], 
         ["2023-12-21T23:52", "2023-12-22T00:03", "40%", "green-green-green-green - all compensation"]]

#This sets the column headers in the spreadsheet
groups = [["Hardpoints",["HP Name", "HP Min", "HP Max"]],
          ["MTMount",["MT Name", "MT Min", "MT Max"]],
          ["Actuator Following Error Summary",["AS ID", "AS P Min", "AS P Max", \
                                              "AS S Min", "AS S Max"]],
          ["Actuator Following Errors",["A ID", "A P Min", "A P Max", \
                                       "A S Min", "A S Max"]]]

addBlankColumn = True # Adds a blank column after each group for readability
firstRow = 2 # Where to place the dataframe in the spreadsheet
firstColumn = 0 # Where to place the dataframe in the spreadsheet

filename = "/home/c/cslage/u/MTM1M3/data/MTM1M3_Summary_26Dec23.xlsx"

# Cells below are the functions that do the work

In [ ]:
def formatSpreadsheet(filename, sheetName, columns):
    workbook = xl.load_workbook(filename=filename)
    sheet = workbook[sheetName]
    for i, column in enumerate(columns):
        columnLetter = xl.utils.get_column_letter(i+1)
        col = sheet.column_dimensions[columnLetter]
        if column == '':
            col.width = 2
        elif 'HP Name' in column:
            col.width = 15
        elif 'HP Min' in column or 'HP Max' in column:
            col.width = 10
            col.number_format = '0.0'
        elif 'ID' in column:
            col.width = 8
            col.number_format = '0'
        else:
            col.width = 8
            col.number_format = '0.0'
    start_column = 1
    for [name, headings] in groups:
        end_column = start_column + len(headings) - 1
        sheet.merge_cells(start_row=2, end_row=2, 
                          start_column=start_column, end_column=end_column)
        cell = sheet.cell(row=2, column=start_column)  
        cell.value = name  
        cell.alignment = xl.styles.Alignment(horizontal='center', vertical='center', wrapText=True)  
        fontStyle = xl.styles.Font(bold=True, size = "10")
        cell.font = fontStyle
        cell.border = xl.styles.Border(outline=True) 
        start_column = end_column + 2
    for heading_column in range(1, end_column + 1):
        cell = sheet.cell(row=3, column=heading_column)
        fontStyle = xl.styles.Font(bold=True, size = "9")
        cell.font = fontStyle
        
    sheet.merge_cells(start_row=1, end_row=1, 
                      start_column=1, end_column=end_column)
    cell = sheet.cell(row=1, column=1)  
    cell.value = sheetName  
    cell.alignment = xl.styles.Alignment(horizontal='center', vertical='center')  
    fontStyle = xl.styles.Font(bold=True, size = "18")
    cell.font = fontStyle
    sheet.row_dimensions[1].height = 30
    sheet.row_dimensions[2].height = 20
    #sheet.print_area = 'A1:O20'
    workbook.save(filename=filename)
    return workbook

In [ ]:
def addColumn(df, columnName, columnData):
    # Allows to add a new column longer or shorter 
    # than the current dataframe
    newLength = len(columnData)
    currentLength = len(df.index)
    #print(columnName, currentLength, newLength)
    if newLength < currentLength:
        columnData.extend(['']*(currentLength - newLength))
    elif currentLength < newLength:
        df = df.reindex(pd.RangeIndex(start=0, stop=newLength), fill_value='')
    df[columnName] = columnData
    return df


In [ ]:
def fivePointStencil(pos, times):
    # Numerically differentiates a data stream
    der = np.zeros(len(pos))
    for i in range(2, len(pos)-2):
        der[i] = pos[i-2] - 8.0*pos[i-1] + 8.0*pos[i+1] - pos[i+2]
        der[i] / 12.0*(times[i] - times[i-1])
    return der


In [ ]:
async def getHardPointData(df, client, start, end):
    ret = await client.select_time_series(
        "lsst.sal.MTM1M3.hardpointActuatorData",
        ["timestamp"]
        + [f"measuredForce{hp}" for hp in range(6)]
        + [f"f{a}" for a in "xyz"]
        + [f"m{a}" for a in "xyz"],
        start,
        end,
        )
    names = []
    mins = []
    maxs = []
    for hp in range(6):
        name = f"measuredForce{hp}"
        data = ret[name]
        min = np.min(data)
        max = np.max(data)
        names.append(name)
        mins.append(min)
        maxs.append(max)
    for a in "xyz":
        name = f"f{a}"
        data = ret[name]
        min = np.min(data)
        max = np.max(data)        
        names.append(name)
        mins.append(min)
        maxs.append(max)
    for a in "xyz":
        name = f"m{a}"
        data = ret[name]
        min = np.min(data)
        max = np.max(data)        
        names.append(name)
        mins.append(min)
        maxs.append(max)

    df = addColumn(df, "HP Name", names)
    df = addColumn(df, "HP Min", mins)
    df = addColumn(df, "HP Max", maxs)
    return df


In [ ]:
async def getFollowingErrors(df, client, start, end):
    ids = []
    primaryMins = []
    primaryMaxs = []
    secondaryMins = []
    secondaryMaxs = []
    forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                     ['*'], start, end)

    for index in range(len(FATable)):
        try:
            id = FATable[index].actuator_id
            fa = force_actuator_from_id(id)
            primary_follow = f"primaryCylinderFollowingError{fa.index}"
            if fa.actuator_type.name == 'DAA':
                secondary_follow = f"secondaryCylinderFollowingError{fa.s_index}"
                secondary_name = fa.orientation.name
            else:
                secondary_follow = None
            primaryMin = np.min(forces[primary_follow].values)
            primaryMins.append(primaryMin)
            primaryMax = np.max(forces[primary_follow].values)
            primaryMaxs.append(primaryMax)
            ids.append(id)
            if secondary_follow:
                secondaryMin = np.min(forces[secondary_follow].values)
                secondaryMins.append(secondaryMin)
                secondaryMax = np.max(forces[secondary_follow].values)
                secondaryMaxs.append(secondaryMax)
            else:
                secondaryMins.append(np.nan)
                secondaryMaxs.append(np.nan)
        except:
            continue
    primaryArgMin = np.argmin(np.array(primaryMins))
    primaryArgMax = np.argmax(np.array(primaryMaxs))
    secondaryArgMin = np.nanargmin(np.array(secondaryMins))
    secondaryArgMax = np.nanargmax(np.array(secondaryMaxs))
    ['' if x is np.nan else x for x in secondaryMins]
    ['' if x is np.nan else x for x in secondaryMaxs]
    df = addColumn(df, "A ID", ids)
    df = addColumn(df, "A P Min", primaryMins)
    df = addColumn(df, "A P Max", primaryMaxs)
    df = addColumn(df, "A S Min", secondaryMins)
    df = addColumn(df, "A S Max", secondaryMaxs)
    #print(secondaryMins)
    #print(secondaryArgMax, secondaryArgMin)
    # Now make the global Min/Max summary
    ids = [ids[primaryArgMin], ids[primaryArgMax],ids[secondaryArgMin],ids[secondaryArgMax]]
    primaryMins = [primaryMins[primaryArgMin],'','','']
    primaryMaxs = ['', primaryMaxs[primaryArgMax],'','']
    secondaryMins = ['','', secondaryMins[secondaryArgMin],'']
    secondaryMaxs = ['','','', secondaryMaxs[secondaryArgMax]]
    df = addColumn(df, "AS ID", ids)
    df = addColumn(df, "AS P Min", primaryMins)
    df = addColumn(df, "AS P Max", primaryMaxs)
    df = addColumn(df, "AS S Min", secondaryMins)
    df = addColumn(df, "AS S Max", secondaryMaxs)
    
    return df


In [ ]:
async def getTMAData(df, client, start, end):
    smoothingFactor = 0.2 # In spline creation
    kernelSize = 100 # In convolution
    kernel = np.ones(kernelSize) / kernelSize

    az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                                ['*'],  start, end)
    el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                                ['*'],  start, end)    
    
    # Now calculates the spline fit and differentiate it to get the acceleration
    azVs = az['actualVelocity'].values
    azXs = az['timestamp'].values - az['timestamp'].values[0]
    azAs = fivePointStencil(azVs, azXs)
    elVs = el['actualVelocity'].values
    elXs = el['timestamp'].values - el['timestamp'].values[0]
    elAs = fivePointStencil(elVs, elXs)
    minAzV = np.min(azVs)
    maxAzV = np.max(azVs)
    minAzA = np.min(azAs)
    maxAzA = np.max(azAs)
    minElV = np.min(elVs)
    maxElV = np.max(elVs)
    minElA = np.min(elAs)
    maxElA = np.max(elAs)
    names = ['Az Vel', 'Az Acc', 'El Vel', 'El Acc']
    mins = [minAzV, minAzA, minElV, minElA]
    maxs = [maxAzV, maxAzA, maxElV, maxElA]
    df = addColumn(df, "MT Name", names)
    df = addColumn(df, "MT Min", mins)
    df = addColumn(df, "MT Max", maxs)
    return df
          

# The cell below creates and formats the spreadsheet

In [ ]:
client = EfdClient("usdf_efd")
for i, [start, end, speed, sheetName] in enumerate(tests):
    start = Time(start, scale='utc')
    end = Time(end, scale='utc')

    columns = []
    for [name, headings] in groups:
        for heading in headings:
            columns.append(heading)
        if addBlankColumn:
            columns.append('')

    df = pd.DataFrame(columns=columns)
    df = await getHardPointData(df, client, start, end)
    df = await getTMAData(df, client, start, end)
    df = await getFollowingErrors(df, client, start, end)
    if i == 0:
        with pd.ExcelWriter(filename) as writer:  
            df.to_excel(writer, sheet_name=sheetName, startrow=firstRow, \
                startcol=firstColumn, index=False)
    else:
        with pd.ExcelWriter(filename, mode='a') as writer:  
            df.to_excel(writer, sheet_name=sheetName, startrow=firstRow, \
                startcol=firstColumn, index=False)

    workbook = formatSpreadsheet(filename, sheetName, columns)
